In [2]:
import matplotlib.pyplot as plt
import numpy as np
from keras.callbacks import EarlyStopping
from keras.layers import Input, Dense
from keras.models import Model, Sequential, load_model
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.backend import mean, square

from spektral.datasets import qm9
from spektral.layers import EdgeConditionedConv, GlobalAttentionPool
from spektral.utils import label_to_one_hot

from os import path
import itertools

Using TensorFlow backend.
/Users/em/anaconda3/envs/venv/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [3]:
A_complete, X_complete, E_complete, y_complete = qm9.load_data(return_type='numpy',
                           nf_keys='atomic_num',
                           ef_keys='type',
                           self_loops=True,
                           amount=None)  # Set to None to train on whole dataset
# one-hot labeling of atoms
uniq_X = np.unique(X_complete)
X_complete = label_to_one_hot(X_complete, uniq_X)

Loading QM9 dataset.
Reading SDF


100%|██████████| 133885/133885 [00:32<00:00, 4074.36it/s]


In [4]:
A, X, E = list(), list(), list()
y = y_complete.sample(10000)
for index, row in y.iterrows():
    A.append(A_complete[index])
    X.append(X_complete[index])
    E.append(E_complete[index])
A = np.stack(A, axis=0)
X = np.stack(X, axis=0)
E = np.stack(E, axis=0)

In [5]:
tasks = list(y.columns)[1:]
y_list = []
for task in tasks:
    y_list.append(y[[task]].values)
for i in range(len(y_list)):
    y_list[i] = StandardScaler().fit_transform(y_list[i]).reshape(-1, y_list[0].shape[-1])

In [6]:
N = X.shape[-2]
F = X.shape[-1] 
S = E.shape[-1]
n_out = y_list[0].shape[-1]
learning_rate = 1e-3
epochs = 10
batch_size = 64
es_patience = 5

In [7]:
n_out

1

In [9]:
optimizer = Adam(lr=learning_rate)
loss = 'mse'

In [10]:
A_train, A_test, \
X_train, X_test, \
E_train, E_test, \
*y_train_test_list = train_test_split(A, X, E, *y_list, test_size = 0.1)

y_train_list = y_train_test_list[::2]
y_test_list = y_train_test_list[1::2]

In [11]:
X_in = Input(shape=(N, F))
A_in = Input(shape=(N, N))
E_in = Input(shape=(N, N, S))

W1215 22:11:03.302671 4754589120 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1215 22:11:03.303588 4754589120 deprecation_wrapper.py:119] From /Users/em/anaconda3/envs/venv/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



In [12]:
def create_single_task_model(X_in, A_in, E_in):
    gc1 = EdgeConditionedConv(64, activation='relu')([X_in, A_in, E_in])
    gc2 = EdgeConditionedConv(128, activation='relu')([gc1, A_in, E_in])
    pool = GlobalAttentionPool(256)(gc2)
    dense1 = Dense(256, activation='relu')(pool)
    output = Dense(n_out)(dense1)
    return Model(inputs=[X_in, A_in, E_in], outputs=output)

In [13]:
def generate_filename(task):
    return path.join('gcnmodels', task + '.h5')

In [14]:
print(tasks)

['A', 'B', 'C', 'mu', 'alpha', 'homo', 'lumo', 'gap', 'r2', 'zpve', 'u0', 'u298', 'h298', 'g298', 'cv', 'u0_atom', 'u298_atom', 'h298_atom', 'g298_atom']


In [18]:
for i in range(0, 15):
    if i == 4:
        continue
    if i == 8:
        continue
    print('learning', tasks[i])
    model = create_single_task_model(X_in, A_in, E_in)
    model.compile(optimizer=optimizer, loss=loss)
    es_callback = EarlyStopping(monitor='val_loss', patience=es_patience)
    model.fit([X_train, A_train, E_train],
             y_train_list[i],
             batch_size=batch_size,
             validation_split=0.1,
             epochs=epochs,
             callbacks=[es_callback])
    model.save_weights(generate_filename(tasks[i]))

learning A
Train on 8100 samples, validate on 900 samples
Epoch 1/10
8100/8100 [==============================] - 48s 6ms/step - loss: 0.2603 - val_loss: 8.5762
Epoch 2/10
8100/8100 [==============================] - 47s 6ms/step - loss: 0.2514 - val_loss: 8.5817
Epoch 3/10
8100/8100 [==============================] - 47s 6ms/step - loss: 0.2487 - val_loss: 8.5866
Epoch 4/10
8100/8100 [==============================] - 48s 6ms/step - loss: 0.2485 - val_loss: 8.5739
Epoch 5/10
8100/8100 [==============================] - 49s 6ms/step - loss: 0.2483 - val_loss: 8.5737
Epoch 6/10
8100/8100 [==============================] - 48s 6ms/step - loss: 0.2482 - val_loss: 8.5688
Epoch 7/10
8100/8100 [==============================] - 49s 6ms/step - loss: 0.2478 - val_loss: 8.5720
Epoch 8/10
8100/8100 [==============================] - 49s 6ms/step - loss: 0.2469 - val_loss: 8.5779
Epoch 9/10
8100/8100 [==============================] - 49s 6ms/step - loss: 0.2473 - val_loss: 8.5760
Epoch 10/10
810

Epoch 6/10
8100/8100 [==============================] - 51s 6ms/step - loss: 0.0712 - val_loss: 0.0636
Epoch 7/10
8100/8100 [==============================] - 50s 6ms/step - loss: 0.0610 - val_loss: 0.0513
Epoch 8/10
8100/8100 [==============================] - 50s 6ms/step - loss: 0.0535 - val_loss: 0.0510
Epoch 9/10
8100/8100 [==============================] - 50s 6ms/step - loss: 0.0493 - val_loss: 0.0466
Epoch 10/10
8100/8100 [==============================] - 50s 6ms/step - loss: 0.0493 - val_loss: 0.0587
learning u0
Train on 8100 samples, validate on 900 samples
Epoch 1/10
8100/8100 [==============================] - 51s 6ms/step - loss: 0.4434 - val_loss: 0.2202
Epoch 2/10
8100/8100 [==============================] - 50s 6ms/step - loss: 0.0724 - val_loss: 0.0268
Epoch 3/10
8100/8100 [==============================] - 50s 6ms/step - loss: 0.0214 - val_loss: 0.0138
Epoch 4/10
8100/8100 [==============================] - 50s 6ms/step - loss: 0.0128 - val_loss: 0.0101
Epoch 5/10
81